## Dependencies

In [1]:
import os
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist

np.random.seed(0)

### MNIST Handwriting

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("Training Data Info")
print("Training Data Shape:", X_train.shape)
print("Training Data Labels Shape:", y_train.shape)

Training Data Info
Training Data Shape: (60000, 28, 28)
Training Data Labels Shape: (60000,)


In [3]:
ndims = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0], ndims)
X_test = X_test.reshape(X_test.shape[0], ndims)
print("Training Shape:", X_train.shape)
print("Testing Shape:", X_test.shape)

Training Shape: (60000, 784)
Testing Shape: (10000, 784)


In [4]:
scaler = MinMaxScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
num_classes = 10
to_categorical(y_train, num_classes)

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

In [6]:
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)
y_train[0]

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32)

# Red neuronal

In [7]:
model = Sequential()

In [8]:
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [9]:
model.add(Dense(100, activation='relu'))

In [10]:
model.add(Dense(num_classes, activation='softmax'))

In [11]:
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [14]:
model.fit(
    X_train,
    y_train,
    epochs=10,
    shuffle=True,
    verbose=2
)

Epoch 1/10
1875/1875 - 1s - loss: 0.2473 - accuracy: 0.9273
Epoch 2/10
1875/1875 - 1s - loss: 0.1092 - accuracy: 0.9667
Epoch 3/10
1875/1875 - 1s - loss: 0.0750 - accuracy: 0.9763
Epoch 4/10
1875/1875 - 1s - loss: 0.0574 - accuracy: 0.9815
Epoch 5/10
1875/1875 - 1s - loss: 0.0456 - accuracy: 0.9848
Epoch 6/10
1875/1875 - 1s - loss: 0.0376 - accuracy: 0.9882
Epoch 7/10
1875/1875 - 1s - loss: 0.0317 - accuracy: 0.9893
Epoch 8/10
1875/1875 - 1s - loss: 0.0256 - accuracy: 0.9910
Epoch 9/10
1875/1875 - 1s - loss: 0.0235 - accuracy: 0.9924
Epoch 10/10
1875/1875 - 1s - loss: 0.0213 - accuracy: 0.9927


In [15]:
# Guardamos el modelo...
model.save("mnist_trained.h5")

In [16]:
# Cargamos el modelo
from tensorflow.keras.models import load_model
model = load_model("mnist_trained.h5")

In [17]:
model_loss, model_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

313/313 - 0s - loss: 0.1015 - accuracy: 0.9757
Loss: 0.10149930417537689, Accuracy: 0.9757000207901001


# Aqui va lo bueno...
------------

In [18]:
import base64, secrets, io, os
from PIL import Image,ImageOps
from urllib.request import urlopen

from flask import Flask, request, jsonify
import numpy as np

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

model = load_model("mnist_trained.h5")

In [19]:
def predecir_im(im, invertir=True):
    image = img_to_array(im)
    image.shape

    # Scale the image pixels by 255 (or use a scaler from sklearn here)
    image /= 255

    # Flatten into a 1x28*28 array 
    img = image.flatten().reshape(-1, 28*28)
    img.shape
    if invertir:
        img = 1 - img
        
    # plt.imshow(img.reshape(28, 28), cmap=plt.cm.Greys)

    resultado = model.predict(img)
    resultado = np.argmax(resultado, axis=-1)
    return int(resultado[0])

In [20]:
imagen64 = "data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAARgAAAEYCAYAAACHjumMAAAAAXNSR0IArs4c6QAAFBJJREFUeF7tnUusJ0UVh38HRp7C4AOIiRrA+NhAjEDcuQKWGrdADBABeaxkozG40KiJIW5khocLiRF3xrgV0bg0PEyAxIQEhkhiYFjAjDLIAHNMkx6YuXPv/VdVV3VXV313e+ucOuc79f+lurqq2sQfBCAAgUIErJBf3EIAAhAQAsMggAAEihFAYIqhxTEEIIDAMAYgAIFiBBCYYmhxDAEIIDCMAQhAoBgBBKYYWhxDAAIIDGMAAhAoRgCBKYYWxxCAAALDGIAABIoRQGCKocUxBCCAwDAGIACBYgQQmGJocQwBCCAwjAEIQKAYAQSmGFocQwACCAxjAAIQKEYAgSmGFscQgAACwxiAAASKEUBgiqHFMQQggMAwBiAAgWIEEJhiaHEMAQggMIwBCECgGAEEphhaHEMAAggMYwACEChGAIEphhbHEIAAAsMYgAAEihFAYIqhxTEEIIDAMAYgAIFiBBCYYmhxDAEIIDCMAQhAoBgBBKYYWhxDAAIIDGMAAhAoRgCBKYYWxxCAAALDGIAABIoRQGCKocUxBCCAwBQYA+7+Y0nflXROAfcpLo9JetzMrksxxgYCqQQQmB3IVSgSqTXeandE0i/M7N5cDvEDgZ0IIDBbyLj7PknfkXRa48NmmNX8FKFpvMoLp9etwDQ8Q4kdUu9JetDM7o41pD0ENhHoQmDc/X5JN1e0JrKpLkv83yU9YGZ3LdE5fbZJoGmBGYXljg4ed3KOzqOSfs6jU06k/fpqVmDc/QlJV/Vb2smZv2Rml072goOuCTQpMOPMhan+9KE9PDb9hNnMdJC9emhVYN6QtLfXohbIexCaP5rZNwv4xmXDBFoVmOEHwV9+Aq+b2cfzu8VjqwQQmPKVPSTpl3M+ZhR+a/aGmX2sPDZ6aIFAqwIz7O3IuVFu1btf3f0Pkr4hKVe92aTXwq9/hhxyDbgZQg3vwt3/KelL4RbbtnxH0q9a2hfi7v+W9KmJXE40H4T8Z3POzjLGjqsZCLQqMNdLejSC36pnKBF5anx8ulXSGTF2G9o+b2ZfzOgPV40QaFJghtq4+32S7tmhTsMi8J97P12c+XX+YTPjzV0jwpArjWYFZhSZYSZzu6SvjsD+LukhM/tdLoBr95N5Q+KwNjMcN+Bc09oHRqb4mxaYTIyad1PgrRMHKJsfNWEJIjBhnLppNZ4y/0GmN04HzezibuCR6CkEEBgGxbYE3P0VSTnEgcemjscYAtNx8Tel7u7PS/r8pnaB/3/FzHK+Ig/slmZLEkBglqS/gr7HR6bvSzo9Q7jcOZMB4ppcIDBrqtaCsWZemxnunBk2MfK2acGaztE1AjMH5Yb6cPfXJV2QKaWnzezKTL5wUyEBBKbCotQekru/IOmyTHE+ZWZcDJYJZm1uEJjaKrKSeDJv0GPfzErqHhsmAhNLjPYfEBg36A2feMmxADz4fdLMrgZxOwQQmHZquVgmo9AMRzL2ZAiCBeAMEGtxgcDUUokG4sj82MRspoExgcA0UMSaUsj82LSPV9k1VTc+FgQmnhkWAQTGfTPfm/jY5GaW82bCgMhpkpMAApOTJr5OIeDuT0n6ygQ0w+7f4dO2d07wgelCBBCYhcD31G2mx6YXzexzPXFrIVcEpoUqriSHDLuA/2Nm568kXcLMdOcHICEQTCCDyHA1ZzDt5Rsyg1m+Bt1F4O4HJF0yIfEDZpbrqMKEMDDdRACB2USI/xchkOF09nNmdnmR4HCajQACkw0ljlIIuPubks5JsZXElZyJ4OYyQ2DmIk0/OxJw9+HzuqmLt4hMxWMLgam4OD2F5u4vSro0MWd2/CaCK22GwJQmjP9gAu7+sqRPBxuc3BCRSQRX0gyBKUkX39EEJn7NgAOS0cTLGiAwZfniPYGAu/9L0mcSTAcTZjKJ4EqYITAlqOJzMoEJ13JyQHIy/XwOEJh8LPGUmYC7vy3pjAS37PZNgFbCBIEpQRWf2Qi4+3Bfb8qVDYhMtiqkO0Jg0tlhOROBCTOZ18zsopnCpJttCCAwDItVEHD3g5IuTAiWRd8EaLlMEJhcJPFTnIC7H5Z0XmRHx8ws11cPIrumOQLDGFgVAXc/lnDNCAcjF6oyArMQeLpNIzDejndXgjWb8BKgTTVBYKYSxH52AhM+j8J6zMzVQmBmBk53eQi4+zOSYu+DYT0mD/5gLwhMMCoa1kYg8ZoHZjEzFhKBmRE2XeUn4O7DZ01i/o6a2ZkxBrRNJ4DApLPDsgICKTfimRnjfqbaAXom0HRThkDKWyUEpkwttvOKwMzHmp4KEYh9q4TAFCrENm4RmPlY01NBAjFvlRCYgoXY4hqBmY81PRUk4O6vSgo62IjAFCwEAjMfXHqah0DM7GWICIGZpy7vs56vK3qCQH4CsesvCEz+GuzmEYGZlze9ZSSQ8gYJgclYgABXCEwAJJrURyBVXCRxZ++M5URgZoRNV3kIxCzobtPjY2Z2XZ5I8LKJAAKziRD/r4bAOGu5c8La4REzO7eahDoIBIHpoMgtpJiymLslb84gLTAQEJgFoNNlHIHY19DbeOcLA3HIs7VGYLKhxFEJAu7+mqRPTvDNou4EeFNNEZipBLEvQsDd90v6zoT1luNxcf9LkQqFOUVgwjjRaiYC40LuICw5vgTAPbwz1W2nbhCYhQtA9x8SyLCQe9zZcAnVfjO7G77LEkBgluVP78PON/f7Jd0uaU8GIG+Z2TkZ/OAiAwEEJgNEXMQTGNdYbpB0frz1jhYHzezijP5wNZEAAjMRIObxBNz9BUmXxVvuaDF8jO0BHokyEs3kCoHJBBI3YQTc/UVJl4a1Dmr1rJldEdSSRrMTQGBmR95nh+Mj0a2Z1lmOQ+QtUeXDCYGpvEBrDs/d90m6RdJZmfN4T9KDPBJlplrAHQJTAGrvLsfZyrclfSQzi+H18yAsw4FH/lZAAIFZQZHWFKK7H5B0SYGYOU9UAGpplwhMacKd+B9nLbdl2oG7ldoBM8v51qmTqiyfJgKzfA1WF8G4Me5GSXsLBz+8fn6IR6LClAu6R2AKwm3R9cTb5EKRHJH0axZxQ3HV2w6Bqbc21UXm7ocy77zdmuPTZnZldYkTUDIBBCYZXR+G49rKTZLOLpjxO5IeZsZSkPBCrhGYhcDX2u1MgnI8/WGNZXjtfFetPIhrGgEEZhq/JqwLHTzcjc1bkh5h8baJ4bNrEghM+zXeNcMCZ4N26u8lM8t5Bqnzyq0jfQRmHXXKEuUCM5Uhbl41Z6neOp0gMOus28aoZ15L2SmeQ2Z2wcZgadAsAQSmsdK6+7BN/0lJn1gwteEw4l/N7NoFY6DrCgggMBUUISWESmYoJ74N4sKnlEI2boPArLDAM+2mDSXDOaFQUh22Q2AWKPp4ludmSWu+nJpXzQuMnbV1icAUqFhljy85Mzws6VH2r+RE2rYvBCahvo3MQEIyR1BCKNFmRwIITMTgGIVluE0Nbpu5DSeify/ph2b20ubmtGiRAD+UwKpm/OpgYI9dN/uvpN9wRmn9YwCBCajhOHPhQF4Aq0JNeFQrBLa0WwQmgLC7vzHD7W0BkXTfZNjAN9xwh9ivZCggMAGFcvfhNnv+6iXAK/NKa4PABBQGgQmAVGcTrt5cuC4ITEABeEQKgFR3k2EG+jhno+YvEgITwJxF3gBI62gyCM1+ruacr1gITCBrXlMHglpHs+GOGg5nzlArBCYC8jiTGT7gfkaEGU3rJcDXIgvXBoEpDLg39+N9ND+S9PWVvNp/18xyf0O7t7LvmC8Cw1CohoC775P0LUkfnTkoZjKFgCMwhcDidhqBBU6k72Pxd1rNtrNGYPIzxWNmAjPNbNzMTsscevfuEJjuh8A6ART6QsJrZnbROonUGTUCU2ddiCqBwPiW78aJi8s8KiWw38kEgckIE1f1EHD3g5IuTIiIR6UEaAhMRmi4WgeBCReEMYvJVGJmMJlA4qZeAu7+duTmSGYxmcqJwGQCiZu6Cbj7cDwgZryzNyZDSWOAZ+gOFxBYhkDigVW++TSxXAjMRICYr4dAwixmSG44FDlc9M5fAgEEJgEaJuskkDiLecvM1vyBvEWLhcAsip/O5yaQ8tldM+N3klgowCWCw2y9BNx9uErz7IgMeG0dAevEpghMIjjM1k3A3d+M+DY4VzoklhuBSQSH2foJRF7m/qSZXb3+rOfNAIGZlze9VUQgUmCGyHlUiqwfAhMJjObtEEjY4fueme1ph0D5TBCY8ozpoVICia+t2RcTUU8EJgIWTdsj4O7D52hjLpo6amZntkeiTEYITBmueF0JgfHiqjtiwmVfTDgtBCacFS0bJeDuL0i6LCI9FnsDYSEwgaBo1jYBdz8s6bzALNkXEwgKgQkERbP2CUS+tmZfTMCQQGACINGkDwKRAjNA4VFpw9BAYPr47ZBlAIHI4wODx2NmdnqA626bIDDdlp7EtxJI3BfDLGaXoYTA8DuDwAkEEvbFHDKzC4C4PQEEhpEBgZMFZr8k9sVkGhUITCaQuGmHQOy+GDbe7Vx7BKad3wWZZCQQc38vAoPAZBx6uOqBQMyCLwKDwPTwmyDHzAQi9sXwJmkH9jwiZR6UuGuHQITAHDGzc9vJPF8mCEw+lnhqjECEwIjHJF5TNzb8Sac0AQRmOmFmMNMZ4qFRAjFHB5jBMINp9GdAWqUIxLxJ4hOzCEypcYjfhglEPCYdNrO9DaNISo1HpCRsGPVCIEJgWOjdZlAgML38UsgziUDM4UfWYU5FjMAkDTuMeiHg7o9JuiYkXwQGgQkZJ7SBwEkEQh+TEBgEhp8OBKIJIDDRyD4w4BEpnR2WnRBAYNILjcCks8OyEwIITHqhEZh0dlh2QiBUYPjKAGswnfwkSDMngQiB4X7eLeCZweQcifhqkkCEwLDZDoFp8jdAUgUJIDDpcJnBpLPDshMCnKpOLzQCk84Oy04IxJyqZrPdyYMCgenkR0Ka0wiEPiYhMAjMtJGGdZcEEJi0sjODSeOGVWcEEJi0giMwadyw6owAApNWcAQmjRtWnRFAYNIKjsCkccOqMwIITFrBEZg0blh1RgCBSSs4ApPGDavOCCAwaQVHYNK4YdUZAQQmreAITBo3rDojgMCkFRyBSeOGVUcE3H2/pDtCUmYn78mUEJiQUUObrgnw6ZL08iMw6eyw7IBAzEHHAQczGGYwHfwsSDEXgZirGiQdNbMzc/Xdgh9mMC1UkRyKEQhd3B0DeMDM7iwWzAodIzArLBohz0PA3V+VdFFobzwenUoKgQkdPbTrikDs2oukY2Z2eleQApJFYAIg0aQ/Au5+bFizjcj8MTO7LqJ9F01jAHYBhCQh4O6HJJ0fQ4LHo+1pITAxo4i2zRNw9zck7Y1M9IiZnRtp00VzBKaLMpNkCAF3f0zSNSFtt7TZZ2Z3J9g1b4LANF9iEgwlEPlK+rjbg2Z2cWgfvbVDYHqrOPmeQsDd/yTp2gQ0bmanJdh1Y4LAdFNqEt2OQORO3a0ueDTaMKwQGH533RJw98OSzksE8KSZXZ1o240ZAtNNqUn0OIFxE93tkvYkUmHmEggOgQkERbM2CLj7E5KumpAN540i4CEwEbBoum4C7v6MpMsnZHHAzC6bYN+dKQLTXcn7S3h8JBpupJvyxuc1Mws++Ngf5e0zRmAYCU0TyPBINPBBXBJHCQKTCA6z+gkknIjeLqnDZhZ7dKB+ODNFiMDMBJpu5iUQc1H3LpExc5lYNgRmIkDM6yMQe1HUDhk8a2ZX1JfduiJCYNZVL6LdhcD4SDRcWTl1XD9tZlcCezqBqYWYHgEeIDCBwPgodJOksye4OdH0OTOb8io7UxhtuEFg2qhjd1mMwnKbpJzXVLL9P/NIQmAyA8VdeQLu/qKkSzP29J6kB7nTJSPR0RUCk58pHgsRKDRrednMPlso5O7dIjDdD4E6ARRYW9kuUa66LFx+BCYBsLtfL+leSV84Yfv5cFH0b5lmJwA9waTQLGW7oNhAN61UQdYITBCmDxu5+32S7tnFbPjcxUN84S8crLvvk3SLpLPCrZJbvjvWhzt0kxGGGyIw4aw0zlweDTRhF+guoGZ6BNoaAW+JAgdvrmYITARJd/+bpK9FmGxq+pakR1qY7SwkGJv4Hv8/b4lCSWVuh8BEAHX3/0k6M8IkpunsYjOKwg2xHxmLSWrhtqyLLVwABCaiAIUFJiISmu5AYJipPNzCjLCVCiMwEZUs8IgU0TtNdyGAsFQ6PBCYiMJELvJGeKbpBAJcYzkBXmlTBCaSsLv/Q9KXI81onp/A7GtW+VNo3yMCk1Bjd39BEpc/J7BLNOERKBHc0mYITGIFxrtHbpZ0TqILzDYTYJaymVHVLRCYDOXJdINahkhW78IlDd8dumv1mZDA+wQQmEwDofKNZpmyLOZmOF7xFzNL+QB9saBwPJ0AAjOdYZSHToSIR5uoUdFuYwRmwdpWJjaIwoJjodWuEZhWK0teEKiAAAJTQREIAQKtEkBgWq0seUGgAgIITAVFIAQItEoAgWm1suQFgQoIIDAVFIEQINAqAQSm1cqSFwQqIIDAVFAEQoBAqwQQmFYrS14QqIAAAlNBEQgBAq0SQGBarSx5QaACAghMBUUgBAi0SgCBabWy5AWBCgggMBUUgRAg0CoBBKbVypIXBCoggMBUUARCgECrBBCYVitLXhCogAACU0ERCAECrRJAYFqtLHlBoAICCEwFRSAECLRKAIFptbLkBYEKCCAwFRSBECDQKgEEptXKkhcEKiCAwFRQBEKAQKsEEJhWK0teEKiAAAJTQREIAQKtEkBgWq0seUGgAgIITAVFIAQItEoAgWm1suQFgQoIIDAVFIEQINAqAQSm1cqSFwQqIIDAVFAEQoBAqwT+D9w1c1UC35THAAAAAElFTkSuQmCC"

In [21]:
from urllib.request import urlopen

imgdata = urlopen(imagen64)
imgdata = imgdata.read()
imgdata = Image.open(io.BytesIO(imgdata))

password_length = 13
extension = "png"
nomre_unico = f'{secrets.token_urlsafe(password_length)}.{extension}'
imgdata = imgdata.resize((28,28))
#imgdata = imgdata.convert("RGB")
imgdata.save(nomre_unico)

im = image.load_img(nomre_unico, color_mode='grayscale',target_size=(28,28))
os.remove(nomre_unico)

predecir_im(im, invertir=False)

3

In [22]:
im